# Large-Scale Data Engineering for AI
Per a fer aquest projecte hem hagafat tres datasets relacionats amb música. Més concretament relacionats amb cançons que te Spotify. A continuació deixem una petita descripció de cadascun:
- **song-lyrics** --> conté unes 25000 cançons amb les seves corresponents lletres.
- **spotify-tracks-dataset** --> conté cançons d'Spotify de 125 gèneres diferents i altres atributs.
- **top-spotify-songs-by-country** --> conté les cançons més populars de 72 països. S'actualitza diàriament.

## The Data Engineering Pipeline
### The Landing Zone
Pel que fa a la landing zone, hem accedit a l'API de Kaggle per a obtenir els datasets. Un pas previ que s'ha de fer per a poder tenir accés és generar un token de Kaggle. Primer has d'entrar en el teu perfil i anar a la configuració, baixant es troba un apartat on hi diu "Create New Token" i d'allà es descarregarà un fitxer .json. Hem utilitzat la [guia](https://www.kaggle.com/docs/api#getting-started-installation-&-authentication) d'ús de l'API per aconseguir-ho. S'hauran de fer aquestes comandes:
```
pip install kaggle
mkdir ~/.kaggle
mv ~/Downloads/kaggle.json ~/.kaggle/kaggle.json
```

Un cop ja tenim accés a la API, hem fet una funció recol·lectora de dades per a baixar els tres datasets i guardar-los a l'adreça desitjada. Aquests datasets estan en format .csv i nosaltres hem preferit passar-los a format .parquet. A l'adreça del dataset de les lletres no només hi ha les lletres sinó que hi ha dos datasets més amb informació sobre els àlbums i més informació sobre les cançons. És per això que només ens hem de quedar amb el de lletres. En conclusió, ens baixem el dataset de l'API, el passem a .parquet, el guardem a la carpeta de "data/landing_zone". Al llarg d'aquesta execució, es van fent moltes comprovacions, en cas que algun moment falli es tingui proouta informació per trobar l'error.

In [ ]:
# Imports and requirements
!pip install kaggle
!pip install pyspark

In [1]:
# Imports and requirements
import os
import datetime
import kaggle
import pandas as pd

In [12]:
# Base directory and Landing Zone directory structure
BASE_DIR = "./data"
LANDING_ZONE_DIR = os.path.join(BASE_DIR, "landing_zone")
os.makedirs(LANDING_ZONE_DIR, exist_ok=True)

kaggle.api.authenticate()

# List of datasets to ingest from Kaggle
datasets = [
        {
            "kaggle_id": "asaniczka/top-spotify-songs-in-73-countries-daily-updated",
            "dataset_name": "top-spotify-songs-by-country",
            "update": True
        },
        {
            "kaggle_id": "maharshipandya/-spotify-tracks-dataset",
            "dataset_name": "spotify-tracks-dataset",
            "update": False
        },
        {
            "kaggle_id": "terminate9298/songs-lyrics",
            "dataset_name": "songs-lyrics.csv",
            "update": False
        }
        ]

In [13]:
# Logging function to timestamp each message
def log(message):
    print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {message}")

def data_collector_kaggle(kaggle_dataset: dict) -> None:
    """
    Downloads a dataset from Kaggle and saves it to the landing zone.

    Parameters:
    kaggle_dataset (dict): A dictionary containing the Kaggle dataset information.
    """

    # Extract dataset information
    kaggle_id = kaggle_dataset["kaggle_id"]
    dataset_name = kaggle_dataset["dataset_name"]
    
    try:  
        log(f"Downloading dataset: {kaggle_id}")

        kaggle.api.dataset_download_files(
            kaggle_id,
            path=LANDING_ZONE_DIR,
            unzip=True
        )

        csv_found = False
        for filename in os.listdir(LANDING_ZONE_DIR):
            if filename in ['songs_details.csv', 'album_details.csv']:
                os.remove(os.path.join(LANDING_ZONE_DIR, filename))
                continue
            if filename.endswith(".csv"):
                csv_path = os.path.join(LANDING_ZONE_DIR, filename)

                # Read CSV with Pandas
                df = pd.read_csv(csv_path)

                # Write Parquet
                final_path = os.path.join(LANDING_ZONE_DIR, f"{dataset_name}.parquet")
                df.to_parquet(final_path, index=False)

                log(f"CSV '{filename}' converted to single Parquet file and saved as '{final_path}'.")

                os.remove(csv_path)
                break  # Stop after processing the first valid CSV
                        
        if not csv_found:
            log(f"No CSV file found in the downloaded dataset. Check the contents of the download.")

    except Exception as e:
        # Log the error
        log(f"Error downloading dataset '{kaggle_id}': {e}")
        return

    # Log the successful download
    log(f"Dataset '{dataset_name}' downloaded successfully.")

def download_and_store_datasets(update: bool = False) -> None:
    """
    Downloads and stores datasets from Kaggle into the landing zone.
    """
    log("Starting the creation of the Landing Zone using Kaggle API")

    for kaggle_dataset in datasets:
        if update and not kaggle_dataset["update"]:
            log(f"Skipping dataset '{kaggle_dataset['dataset_name']}' as update is set to False.")
            continue
        try:
            dataset_name = kaggle_dataset["dataset_name"]
            data_collector_kaggle(kaggle_dataset)
            log(f"Dataset '{dataset_name}' processed successfully.")
        except Exception as e:
            log(f"Error processing dataset '{dataset_name}': {e}")

    log("All datasets have been processed.")
    log("Landing Zone creation completed.")

download_and_store_datasets(update=False)

[2025-04-12 13:35:09] Starting the creation of the Landing Zone using Kaggle API
[2025-04-12 13:35:09] Downloading dataset: asaniczka/top-spotify-songs-in-73-countries-daily-updated
Dataset URL: https://www.kaggle.com/datasets/asaniczka/top-spotify-songs-in-73-countries-daily-updated
[2025-04-12 13:35:21] CSV 'universal_top_spotify_songs.csv' converted to single Parquet file and saved as './data/landing_zone/top-spotify-songs-by-country.parquet'.
[2025-04-12 13:35:21] No CSV file found in the downloaded dataset. Check the contents of the download.
[2025-04-12 13:35:21] Dataset 'top-spotify-songs-by-country' downloaded successfully.
[2025-04-12 13:35:21] Dataset 'top-spotify-songs-by-country' processed successfully.
[2025-04-12 13:35:21] Downloading dataset: maharshipandya/-spotify-tracks-dataset
Dataset URL: https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset
[2025-04-12 13:35:23] CSV 'dataset.csv' converted to single Parquet file and saved as './data/landing_zone/sp

### The Formatted Zone

### The Trusted Zone

### The Exploitation Zone

## The Data Analysis Pipelines